In [45]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import json
import numpy as np

In [17]:
csv_tickets_1 = "Resources/parking_tickets_2015/Parking_Tags_Data_2015_1.csv"
ticket_df_1 = pd.read_csv(csv_tickets_1)
#ticket_df_1["date_of_infraction"] = ticket_df_1["date_of_infraction"].astype(int)
ticket_df_1.head()

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***87325,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1.0,OPP,11 CENTRE AVE,NaN,NaN,ON
1,***87326,20150101,15,PARK-WITHIN 3M OF FIRE HYDRANT,100,1.0,OPP,11 CENTRE AVE,NaN,NaN,ON
2,***09493,20150101,3,PARK ON PRIVATE PROPERTY,30,2.0,NR,2425 JANE ST,NaN,NaN,ON
3,***67358,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3.0,N/S,DUNDAS SQ,E/O,YONGE ST,ON
4,***87327,20150101,5,PARK-SIGNED HWY-PROHIBIT DY/TM,40,3.0,N/S,ARMOURY ST,W/O,CENTRE AVE,ON


In [18]:
csv_tickets_2 = "Resources/parking_tickets_2015/Parking_Tags_Data_2015_2.csv"
ticket_df_2 = pd.read_csv(csv_tickets_2)
ticket_df_2.head()

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***26271,20150510,29,PARK PROHIBITED TIME NO PERMIT,30,219.0,NR,21 CLAXTON BLVD,NaN,NaN,ON
1,***47587,20150510,29,PARK PROHIBITED TIME NO PERMIT,30,219.0,NR,209 FAIRVIEW AVE,NaN,NaN,ON
2,***95886,20150510,3,PARK ON PRIVATE PROPERTY,30,220.0,AT,24 MABELLE AVE,NaN,NaN,ON
3,***60513,20150510,2,PARK - LONGER THAN 3 HOURS,15,220.0,NR,18 LINDEN AVE,NaN,NaN,ON
4,***00993,20150510,8,STAND VEH.-PROHIBIT TIME/DAY,60,220.0,NR,364 ADELAIDE ST W,NaN,NaN,ON


In [19]:
csv_tickets_3 = "Resources/parking_tickets_2015/Parking_Tags_Data_2015_3.csv"
ticket_df_3 = pd.read_csv(csv_tickets_3)
ticket_df_3.head()

,tag_number_masked,date_of_infraction,infraction_code,infraction_description,set_fine_amount,time_of_infraction,location1,location2,location3,location4,province
0,***98714,20150911,210,PARK FAIL TO DISPLAY RECEIPT,30,1707.0,NR,151 DAN LECKIE WAY,NaN,NaN,ON
1,***03276,20150911,403,STOP-SIGNED HIGHWAY-RUSH HOUR,150,1707.0,NR,2143 DANFORTH AVE,NaN,NaN,ON
2,***06931,20150911,2,PARK - LONGER THAN 3 HOURS,15,1707.0,NR,90 BURNSIDE DR,NaN,NaN,ON
3,***78878,20150911,29,PARK PROHIBITED TIME NO PERMIT,30,1707.0,NR,41 DALTON RD,NaN,NaN,ON
4,***01730,20150911,210,PARK FAIL TO DISPLAY RECEIPT,30,1707.0,NR,10 ELLERBECK ST,NaN,NaN,ON


In [20]:
tickets_2015_df=pd.concat([ticket_df_1,ticket_df_2,ticket_df_3])
locations = list(tickets_2015_df["location2"])
locations

['11 CENTRE AVE',
 '11 CENTRE AVE',
 '2425 JANE ST',
 'DUNDAS SQ',
 'ARMOURY ST',
 '46 SPADINA AVE',
 '39 LAKEVIEW AVE',
 'ARMOURY ST',
 'ARMOURY ST',
 'FENWICK AVE',
 '46 SPADINA AVE',
 '21 CAMBRIDGE AVE',
 'ARMOURY ST',
 'ARMOURY ST',
 'FENWICK AVE',
 '40 HIGH PARK AVE',
 'ARMOURY ST',
 'FENWICK AVE',
 'FENWICK AVE',
 '46 SPADINA AVE',
 '30 KINGSWOOD RD',
 '27 CAMBRIDGE AVE',
 '145 STRATHMORE BLVD',
 'ARMOURY ST',
 'FENWICK AVE',
 'KINGSWOOD RD',
 'KINGSWOOD RD',
 '49 CAMBRIDGE AVE',
 '2 CAITHNESS AVE',
 '19 MACEY AVE',
 '10 YUKON PLACE',
 '40 HIGH PARK AVE',
 'ARMOURY ST',
 'FENWICK AVE',
 '60 KINGSWOOD RD',
 '6 CAITHNESS AVE',
 '40 HIGH PARK AVE',
 'ARMOURY ST',
 'ARMOURY ST',
 '380 WELLINGTON ST W',
 '60 KINGSWOOD RD',
 '49 CAMBRIDGE AVE',
 '87 STRATHMORE BLVD',
 'ALBION AVE',
 '14 LONDON GREEN CT',
 'FENWICK AVE',
 'FENWICK AVE',
 '62 KINGSWOOD RD',
 '85 STRATHMORE BLVD',
 '178 DUPONT ST',
 '70 HIGH PARK AVE',
 '111 CHESTNUT ST',
 '5351 YONGE ST',
 '80 LYNN WILLIAMS ST',
 '63 CAM

In [21]:

parking_id = []
address = []
rate = []
with open("Resources/greenPParking2015.json") as json_file:  
    data = json.load(json_file)
    for p in data['carparks']:
        parking_id.append(p["id"])
        address.append(p["address"])
        rate.append(p["rate"])
parking_df=pd.DataFrame({"parking_id":parking_id,
                        "address":address,
                        "rate":rate})
#parking_df["address"].value_counts()

In [22]:
#store addresses in python list 
address_list_complete = list(parking_df["address"])



In [23]:
#Upper Case Addresses
address_list_complete = [address.upper() for address in address_list_complete]
address_list_complete

['20 CHARLES STREET',
 '13 ISABELLA STREET',
 '15 WELLESLEY STREET EAST',
 '21 PLEASANT BLVD.',
 '30 ALVIN AVENUE',
 '20 DELISLE AVENUE',
 '37 YORKVILLE AVENUE',
 '670 PAPE AVENUE',
 '351 KEELE STREET',
 '385 PACIFIC AVENUE',
 '101 CEDARVALE AVENUE',
 '72 AMROTH AVENUE',
 '37 QUEEN STREET EAST',
 '716 PAPE AVENUE',
 '75 HOLLY STREET',
 '45 BAY STREET',
 '25 DUNDAS STREET EAST',
 '110 QUEEN STREET WEST (NATHAN PHILLIPS SQUARE GARAGE)',
 '20 CASTLEFIELD AVENUE',
 '7 NORTON AVENUE',
 '91 VIA ITALIA',
 '2 CHURCH STREET (ST. LAWRENCE GARAGE)',
 '14 FULLER AVENUE',
 '111 BROADVIEW AVENUE',
 '125 BURNABY BLVD.',
 '85 LEE AVENUE',
 '365 LIPPINCOTT STREET',
 '40 YORK STREET',
 '803 RICHMOND STREET WEST',
 '23 BEDFORD PARK AVENUE WEST',
 '9 BEDFORD ROAD (HOTEL INTERCONTINENTAL)',
 '1119 QUEEN STREET WEST',
 '265 DURIE STREET',
 '20 ST. ANDREW STREET (KENSINGTON GARAGE)',
 '35 BELLEVUE AVENUE',
 '35 ERINDALE AVENUE',
 '405 SHERBOURNE STREET',
 '400 KEELE STREET',
 '695 LANSDOWNE AVENUE',
 '92 MAR

In [24]:
#Removing dots from address
no_dot_address = []
for address in address_list_complete:
    dummy = ""
    for char in address:
        if char!=".":
            dummy=dummy+char
    no_dot_address.append(dummy) 
address_list_complete = no_dot_address
len(address_list_complete)

239

In [25]:
#Filter through list and save items as seperate lists 
street="STREET"
#blvd_dot="BLVD."

blvd="BLVD"
avenue="AVENUE"
road="ROAD"
dr = " DR"
#dr_dot = "DR."
#ave_dot="AVE."
circle="CIRCLE"
lane=" LANE"
drive=" DRIVE"
commuter_lot = "COMMUTER LOT"
east = "EAST"
west = "WEST"

#street_address = [address for address in address_list if street in address and commuter_lot not in address]
#blvd_dot_address = [address for address in address_list if blvd_dot in address and commuter_lot not in address]
#blvd_address = [address for address in address_list if blvd in address and commuter_lot not in address]
#avenues_address = [address for address in address_list if avenue in address and commuter_lot not in address]
#road_address =[address for address in address_list if road in address and commuter_lot not in address]
#dr_address = [address for address in address_list if dr in address and commuter_lot not in address]
#dr_dot_address = [address for address in address_list if blvd_dot in address and commuter_lot not in address]
#ave_dot_address = [address for address in address_list if ave_dot in address and commuter_lot not in address]
#circle_address = [address for address in address_list if circle in address and commuter_lot not in address]
#lane_address = [address for address in address_list if lane in address and commuter_lot not in address]
#drive_address = [address for address in address_list if drive in address and commuter_lot not in address]
#commuter_lot_address = [address for address in address_list if commuter_lot in address]
#length = len(street_address)+len(blvd_address)+len(avenues_address)+\
#        len(road_address)+len(dr_address)+len(circle_address)+len(lane_address)+\
#        len(drive_address)+len(commuter_lot_address)
#print(length)
#print(len(address_list))

In [26]:
#Clean up addresses to same formatting (All Upper Case, using proper shorthand callings)
streets = []

for address in address_list_complete:
    split_address = address.split()
    #print(split_address)
    dumy=""
    for word in split_address:
            
        if word == east:
            word = "E"
            dumy = dumy+ " " + word
        elif word == west:
            word = "W"
            dumy=dumy+ " " + word
        elif word == street:
            #print("street")
            word = "ST"
            dumy = dumy + " "+ word
        elif word == blvd:
            word = "BLVD"
            dumy=dumy +" " + word
        elif word == avenue:
            word = "AVE"
            dumy=dumy +" " + word
        elif word == road:
            word = "RD"
            dumy=dumy +" " + word
        elif word == dr:
            word = "DR"
            dumy=dumy +" " + word
        elif word == circle:
            word = "CRCL"
            dumy=dumy +" " + word
        elif word == lane:
            word = "LANE"
            dumy=dumy +" " + word
        elif word == drive:
            word = "DRIVE"
            dumy=dumy +" " + word
        elif dumy == "":
            dumy = word
        else:
            dumy = dumy +" " + word
    streets.append(dumy)

address_list_complete = streets
len(address_list_complete)

239

In [27]:
#This Cell just to check if there any common addresses
common_address_locations = []
common_address_address_list = []
for address in address_list_complete:
    for location in locations:
        if str(location) in address and str(location) not in common_address_locations:
            common_address_locations.append(location)
            common_address_address_list.append(address)
            
                

In [16]:
#We found 644 common entries
print(len(common_address_locations))
print(len(common_address_address_list))

644
644


In [38]:
#print(common_address_locations)


In [39]:
#print(common_address_address_list)

In [29]:
#create location2 column in parking_df using the cleaned up address list
parking_df["location2"] = address_list_complete
#parking_df.drop(labels = "Clean Address",axis=1,inplace=True)

In [30]:
parking_df.head()

,parking_id,address,rate,location2
0,1,20 Charles Street,$2.00 / Half Hour,20 CHARLES ST
1,3,13 Isabella Street,$2.25 / Half Hour,13 ISABELLA ST
2,5,15 Wellesley Street East,$2.00 / Half Hour,15 WELLESLEY ST E
3,11,21 Pleasant Blvd.,$1.75 / Half Hour,21 PLEASANT BLVD
4,12,30 Alvin Avenue,$2.50 / Half Hour,30 ALVIN AVE


In [31]:
#Merge the parking_df and tickets_2015_df on column labeled location2

merged_df = pd.merge(parking_df,tickets_2015_df,on="location2")
merged_df.dtypes

parking_id                 object
address                    object
rate                       object
location2                  object
tag_number_masked          object
date_of_infraction          int64
infraction_code             int64
infraction_description     object
set_fine_amount             int64
time_of_infraction        float64
location1                  object
location3                  object
location4                  object
province                   object
dtype: object

In [56]:
#Take only the columns we want to display and store them in merged_df
merged_df = merged_df[["parking_id","rate","location2","infraction_description","set_fine_amount"]]
len(merged_df)

16394

In [65]:
#Rename the Dataframe index as 'id' as to match the primary key of the SQL database
#A list was created using numpy, and then inserted into the Dataframe as the index
index = np.arange(0,len(merged_df))
merged_df["id"] = index
merged_df.set_index("id",inplace=True)
merged_df.rename(columns = {"location2":"clean_address"},inplace=True)
merged_df.dtypes

parking_id                object
rate                      object
clean_address             object
infraction_description    object
set_fine_amount            int64
dtype: object

In [61]:
#Create SQL connection
rds_connection_string = "root:root123@127.0.0.1:3306/parking_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [62]:
#Check for table
engine.table_names()

['parking_tickets']

In [66]:
#Export to SQL. If table already exists, data will be appended
merged_df.to_sql(name='parking_tickets', con=engine, if_exists='append', index=True)